In [28]:
!pip install yfinance mplfinance timm

In [29]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
from sklearn import model_selection
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import timm
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torch.utils.data import DataLoader
import time
import copy
from datetime import datetime, timedelta
import os
from datetime import datetime

In [99]:


# 現在の日付と時刻を取得
current_datetime = datetime.now()

# 日付を 'yyyy-mm-dd' 形式の文字列にフォーマット
formatted_date = current_datetime.strftime('%Y-%m-%d')

print(formatted_date)

2024-01-22


In [104]:
# Get the current date and format the date into a string in 'yyyy-mm-dd' format
current_date = datetime.now().strftime('%Y-%m-%d')

data = yf.download('EURJPY=X', start='2018-01-03', end=current_date)

#data = yf.download('EURJPY=X', start='2018-01-03', end='2024-01-21')

data.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-15,158.865005,159.824005,158.822998,158.863998,158.863998,0
2024-01-16,159.563004,160.000000,159.242996,159.563004,159.563004,0
2024-01-17,160.179993,161.214996,160.018005,160.179993,160.179993,0
2024-01-18,161.223007,161.367004,160.656006,161.223007,161.223007,0
2024-01-19,161.126999,161.839005,160.807007,161.126999,161.126999,0


In [31]:
data['Up'] = 0
# "future_num" days after the close predicted up or down
future_num = 1

for i in range(len(data)-future_num):
    if data['Close'][i] < data['Close'][i+future_num]:
        data['Up'][i] = 1
"""
data['Up'] = 0
data['Up'][data['Close'] < data['Close'].shift(-future_num)] = 1
"""

<ipython-input-31-cf3a3126341f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Up'][i] = 1


"\ndata['Up'] = 0\ndata['Up'][data['Close'] < data['Close'].shift(-future_num)] = 1\n"

In [32]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,Up
Date,,,,,,,
2018-01-03,135.406998,135.496002,134.809998,135.401001,135.401001,0,0
2018-01-04,135.253998,136.352997,135.244003,135.233994,135.233994,0,1
2018-01-05,136.093002,136.626007,136.089996,136.121994,136.121994,0,1
2018-01-08,136.188995,136.313995,135.130005,136.203003,136.203003,0,0
2018-01-09,135.460999,135.468002,134.056000,135.434006,135.434006,0,0


In [33]:
# Set path to save train images
folder_path = './DataFolder'

# If the directory does not exist, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"The directory '{folder_path}' has been created!")
else:
    print(f"The directory '{folder_path}' has already exist!")

The directory './DataFolder' has already exist!


In [34]:
data['ImageName'] =  [folder_path +'/'+ str(data.index[i].year)+ '-'  + str(data.index[i].month) + '-' + str(data.index[i].day) + '.png' for i in range(len(data))]

In [35]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume,Up,ImageName
Date,,,,,,,,
2024-01-15,158.865005,159.824005,158.822998,158.863998,158.863998,0,1,./DataFolder/2024-1-15.png
2024-01-16,159.563004,160.000000,159.242996,159.563004,159.563004,0,1,./DataFolder/2024-1-16.png
2024-01-17,160.179993,161.214996,160.018005,160.179993,160.179993,0,1,./DataFolder/2024-1-17.png
2024-01-18,161.223007,161.367004,160.656006,161.223007,161.223007,0,0,./DataFolder/2024-1-18.png
2024-01-19,161.126999,161.839005,160.807007,161.126999,161.126999,0,0,./DataFolder/2024-1-19.png


In [36]:
from tqdm import tqdm
# Moving Average Calculation
short_window = 5   # 5-day moving average
long_window = 20   # 20-day moving average
data['SMA5'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
data['SMA20'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

# Chart Style Setting
mc = mpf.make_marketcolors(up='#77d879', down='#db3f3f', inherit=True)
s = mpf.make_mpf_style(base_mpf_style='nightclouds', marketcolors=mc, gridstyle='')

figsize = 2.56, 2.56
# Generate Charts
seq_len = 20
for i in tqdm(range(len(data) - seq_len + 1)):
    c = data.iloc[i:i + seq_len]

    # Additional plots of moving average line
    add_plot = [
        mpf.make_addplot(c['SMA5'], color='blue', alpha=0.5),
        mpf.make_addplot(c['SMA20'], color='red', alpha=0.5),
    ]
    """
    fig, ax = mpf.plot(c, type='candle', style=s, addplot=add_plot, returnfig=True)
    # Axis setting
    ax[0].set_xticklabels([])
    ax[0].set_yticklabels([])
    ax[0].xaxis.set_visible(False)
    ax[0].yaxis.set_visible(False)
    """
    fig, ax = mpf.plot(c, type='candle', style=s, addplot=add_plot, figsize=figsize, returnfig=True)

    # Axis setting
    ax[0].set_xticklabels([])
    ax[0].set_yticklabels([])
    ax[0].xaxis.set_visible(False)
    ax[0].yaxis.set_visible(False)

    # Save file as an image
    pngfile = data['ImageName'][i + seq_len - 1]
    fig.savefig(pngfile, pad_inches=0.1, transparent=False,bbox_inches='tight')
    plt.close(fig)

print("Candlestick charts and moving averages have been successfully generated!")

100%|██████████| 1559/1559 [03:57<00:00,  6.56it/s]

Candlestick charts and moving averages have been successfully generated!


In [37]:
data

,Open,High,Low,Close,Adj Close,Volume,Up,ImageName,SMA5,SMA20
Date,,,,,,,,,,
2018-01-03,135.406998,135.496002,134.809998,135.401001,135.401001,0,0,./DataFolder/2018-1-3.png,135.401001,135.401001
2018-01-04,135.253998,136.352997,135.244003,135.233994,135.233994,0,1,./DataFolder/2018-1-4.png,135.317497,135.317497
2018-01-05,136.093002,136.626007,136.089996,136.121994,136.121994,0,1,./DataFolder/2018-1-5.png,135.585663,135.585663
2018-01-08,136.188995,136.313995,135.130005,136.203003,136.203003,0,0,./DataFolder/2018-1-8.png,135.739998,135.739998
2018-01-09,135.460999,135.468002,134.056000,135.434006,135.434006,0,0,./DataFolder/2018-1-9.png,135.678799,135.678799
...,...,...,...,...,...,...,...,...,...,...
2024-01-15,158.865005,159.824005,158.822998,158.863998,158.863998,0,1,./DataFolder/2024-1-15.png,158.778201,157.286417
2024-01-16,159.563004,160.000000,159.242996,159.563004,159.563004,0,1,./DataFolder/2024-1-16.png,159.104001,157.469917
2024-01-17,160.179993,161.214996,160.018005,160.179993,160.179993,0,1,./DataFolder/2024-1-17.png,159.562799,157.579066


In [38]:
import numpy as np
#x_train, x_val, y_train, y_val = model_selection.train_test_split(data['ImageName'].values, data['Up'].values, test_size=0.1,random_state=1)
num_val = 10
val_start_index = len(data) - num_val

# Split into training and validation data
x_train = data['ImageName'].values[:val_start_index]
y_train = data['Up'].values[:val_start_index]
z_train = np.array(data.index[:val_start_index].strftime('%Y-%m-%d').tolist())
x_val = data['ImageName'].values[val_start_index:]
y_val = data['Up'].values[val_start_index:]
z_val = np.array(data.index[val_start_index:].strftime('%Y-%m-%d').tolist())


In [39]:
z_val

array(['2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11',
       '2024-01-12', '2024-01-15', '2024-01-16', '2024-01-17',
       '2024-01-18', '2024-01-19'], dtype='<U10')

In [40]:
y_val

array([0, 0, 1, 0, 0, 1, 1, 1, 0, 0])

In [41]:
X = []
Y = []
Z = []
X_val = []
Y_val = []
Z_val = []
transform = transforms.Compose([transforms.ToTensor()])

#for image_file,label in zip(x_train,y_train):
for image_file,label,date in zip(x_train,y_train,z_train):
    try:
        image = Image.open(image_file)
        image = image.convert('RGB')
        image = transform(image)
        #image = np.asarray(image)
        X.append(image)
        Y.append(label)
        Z.append(date)
    except FileNotFoundError as e:
        print(e)


for image_file,label,date in zip(x_val,y_val,z_val):
    try:
        image = Image.open(image_file)
        image = image.convert('RGB')
        image = transform(image)
        X_val.append(image)
        Y_val.append(label)
        Z_val.append(date)
    except FileNotFoundError as e:
        print(e)

[Errno 2] No such file or directory: './DataFolder/2018-1-3.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-4.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-5.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-8.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-9.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-10.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-11.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-12.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-15.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-16.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-17.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-18.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-19.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-22.png'
[Errno 2] No such file or directory: './DataFolder/2018-1-23.png'
[Errno 2] No su

In [42]:
import numpy as np
# Set directory path
folder_path = './data'

# If the directory does not exist, create it.
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"The directory '{folder_path}' has been created!")
else:
    print(f"The directory '{folder_path}' has already exist!")

# Save as numpy array
np.save('./data/x_train.npy', x_train)
np.save('./data/x_val.npy', x_val)
np.save('./data/y_train.npy', y_train)
np.save('./data/y_val.npy', y_val)
np.save('./data/z_train.npy', z_train)
np.save('./data/z_val.npy', z_val)

The directory './data' has already exist!


In [43]:
class EUR_Dataset(Dataset):
    def __init__(self, image, label_list,date_list, phase=None):

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(256)
        ])
        self.image = image
        self.label_list = torch.tensor(label_list,dtype=torch.long)
        self.phase = phase
        self.date_list = date_list

    def __getitem__(self,index):
        # Load and preprocess the index-th image
        image = self.image[index]
        # Get the index-th label
        label = self.label_list[index]
        date = self.date_list[index]
        return image, label, date

    def __len__(self):
        return len(self.image)

In [44]:
train_dataset = EUR_Dataset(X,Y,Z,phase='train')
total_samples = len(train_dataset)
print(total_samples)
val_dataset = EUR_Dataset(X_val,Y_val,Z_val,phase='val')
val_samples = len(val_dataset)
print(val_samples)

1549
10


In [81]:

#model = models.resnet18(pretrained=True)
model = models.resnet50(pretrained=False)
#model = models.resnet101(pretrained=True)
torch.cuda.empty_cache()
num_ftrs = model.fc.in_features

model.fc  = nn.Sequential(
                      nn.Linear(num_ftrs ,2))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [82]:
from torch.optim import lr_scheduler

In [83]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

criterion = nn.CrossEntropyLoss()


# Observe that all parameters are being optimized
optimizer_ft = optim.AdamW(model.parameters())  # Initial learning rate set to 0.01
num_epochs = 40
# Defining LambdaLR


batchsize = 64

loss_list = []
val_loss_list = []
# Create instance of DataLoaders
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsize, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batchsize, shuffle=False)

# Store data loader as a dictionary
dataloaders_dict = {
    'train' : train_dataloader,
    'val' : val_dataloader
}

cuda:0


In [84]:
Train_loss = []
Val_loss = []
Train_acc = []
Val_acc = []

"""
def train_model(model, criterion, optimizer,  num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    #best_acc = 0.0
    best_loss = 1000.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        lambda1 = lambda epoch: 0.1 if epoch >= num_epochs // 2 else 1.0  # num_epochs is the total number of epochs
        scheduler = lr_scheduler.LambdaLR(optimizer_ft, lr_lambda=lambda1)
        # Each epoch has a training and validation phase

        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0

            # Switching between learning mode and validation mode
            if phase == 'train':
                model.train()
            else:
                model.eval()

            #for i,data in enumerate(trainloader):
            i = 0
            data_num = 0
            for inputs, labels, dates in dataloaders_dict[phase]:
                i += 1
                #for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, axis=1)

                    #print(preds)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    data_num += inputs.size(0)

            epoch_loss = running_loss/data_num
            epoch_acc = running_corrects.double()/data_num


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'train':
                Train_loss.append(epoch_loss)
                Train_acc.append(epoch_acc)
            else:
                Val_loss.append(epoch_loss)
                Val_acc.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
"""

"\ndef train_model(model, criterion, optimizer,  num_epochs=25):\n    since = time.time()\n\n    best_model_wts = copy.deepcopy(model.state_dict())\n    #best_acc = 0.0\n    best_loss = 1000.0\n\n    for epoch in range(num_epochs):\n        print('Epoch {}/{}'.format(epoch, num_epochs - 1))\n        print('-' * 10)\n        lambda1 = lambda epoch: 0.1 if epoch >= num_epochs // 2 else 1.0  # num_epochs is the total number of epochs\n        scheduler = lr_scheduler.LambdaLR(optimizer_ft, lr_lambda=lambda1)\n        # Each epoch has a training and validation phase\n\n        for phase in ['train', 'val']:\n            running_loss = 0.0\n            running_corrects = 0\n\n            # Switching between learning mode and validation mode\n            if phase == 'train':\n                model.train()\n            else:\n                model.eval()\n\n            #for i,data in enumerate(trainloader):\n            i = 0\n            data_num = 0\n            for inputs, labels, dates in

In [85]:
from torch.cuda.amp import GradScaler, autocast

# Create an instance of GradScaler
scaler = GradScaler()

def train_model(model, criterion, optimizer,  num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    #best_acc = 0.0
    best_loss = 1000.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        lambda1 = lambda epoch: 0.1 if epoch >= num_epochs // 2 else 1.0  # num_epochs is the total number of epochs
        scheduler = lr_scheduler.LambdaLR(optimizer_ft, lr_lambda=lambda1)
        # Each epoch has a training and validation phase

        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0

            # Switching between learning mode and validation mode
            if phase == 'train':
                model.train()
            else:
                model.eval()

            #for i,data in enumerate(trainloader):
            i = 0
            data_num = 0
            for inputs, labels, dates in dataloaders_dict[phase]:
                i += 1
                #for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                #optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase=='train'):
                    with autocast():
                      outputs = model(inputs)
                      preds = torch.argmax(outputs, axis=1)

                      #print(preds)
                      loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                      scaler.scale(loss).backward()  # Scaling loss
                      scaler.step(optimizer)  # Optimizer Step
                      scaler.update()  # Update GradScaler
                      optimizer.zero_grad()  # Reset slope to 0
                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    data_num += inputs.size(0)

            epoch_loss = running_loss/data_num
            epoch_acc = running_corrects.double()/data_num


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'train':
                Train_loss.append(epoch_loss)
                Train_acc.append(epoch_acc)
            else:
                Val_loss.append(epoch_loss)
                Val_acc.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [86]:
model = train_model(model,criterion,optimizer_ft,num_epochs=num_epochs)

Epoch 0/39
----------
train Loss: 0.9014 Acc: 0.5094
val Loss: 1.7773 Acc: 0.4000

Epoch 1/39
----------
train Loss: 0.7340 Acc: 0.5152
val Loss: 0.7596 Acc: 0.6000

Epoch 2/39
----------
train Loss: 0.6684 Acc: 0.6217
val Loss: 0.6335 Acc: 0.7000

Epoch 3/39
----------
train Loss: 0.7229 Acc: 0.6075
val Loss: 0.8624 Acc: 0.4000

Epoch 4/39
----------
train Loss: 0.5140 Acc: 0.7566
val Loss: 1.5092 Acc: 0.6000

Epoch 5/39
----------
train Loss: 0.4900 Acc: 0.7631
val Loss: 1.3747 Acc: 0.4000

Epoch 6/39
----------
train Loss: 0.3805 Acc: 0.8302
val Loss: 1.0661 Acc: 0.4000

Epoch 7/39
----------
train Loss: 0.3310 Acc: 0.8606
val Loss: 0.5566 Acc: 0.7000

Epoch 8/39
----------
train Loss: 0.3682 Acc: 0.8341
val Loss: 0.7098 Acc: 0.5000

Epoch 9/39
----------
train Loss: 0.2515 Acc: 0.9051
val Loss: 0.2435 Acc: 0.9000

Epoch 10/39
----------
train Loss: 0.2151 Acc: 0.9128
val Loss: 0.6176 Acc: 0.7000

Epoch 11/39
----------
train Loss: 0.1899 Acc: 0.9264
val Loss: 0.9195 Acc: 0.6000

Ep

In [91]:
torch.save(model.state_dict(), 'data/model.pth')

In [93]:
model.load_state_dict(torch.load('data/model.pth'))

# Set model in evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [94]:
# Record prediction results and prediction probabilities
val_dataloader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)
pred_list = np.empty([1, 2])

i = 0
input_list = []
output_list = []
sample_list = []
for inputs, labels,dates in val_dataloader:
    i += 1
    #for inputs, labels in dataloaders[phase]:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        sm = nn.Softmax(dim=1)
        outputs2 = np.array(sm(outputs.cpu()))
        pred_list = np.vstack((pred_list,outputs2))
        output_list.append(outputs2)
        input_list.append(labels)
        labels_np = labels.cpu().numpy()  # Convert tensor to NumPy array
        sample_list.append([labels_np,outputs2,dates])
print(sample_list)

[[array([0]), array([[0.9245624 , 0.07543759]], dtype=float32), ('2024-01-08',)], [array([0]), array([[0.44457436, 0.5554256 ]], dtype=float32), ('2024-01-09',)], [array([1]), array([[2.7710556e-05, 9.9997234e-01]], dtype=float32), ('2024-01-10',)], [array([0]), array([[0.74768925, 0.25231084]], dtype=float32), ('2024-01-11',)], [array([0]), array([[0.86840004, 0.13159993]], dtype=float32), ('2024-01-12',)], [array([1]), array([[5.019836e-05, 9.999498e-01]], dtype=float32), ('2024-01-15',)], [array([1]), array([[0.07472146, 0.9252785 ]], dtype=float32), ('2024-01-16',)], [array([1]), array([[0.00778029, 0.9922197 ]], dtype=float32), ('2024-01-17',)], [array([0]), array([[0.9592106 , 0.04078941]], dtype=float32), ('2024-01-18',)], [array([0]), array([[0.9756271 , 0.02437287]], dtype=float32), ('2024-01-19',)]]


In [95]:
data = {'Label': [], 'Pred_down': [], 'Pred_up': [], 'Date': []}

# Analyze each element of sample_list and add it to the data list
for item in sample_list:
    data['Label'].append(item[0][0])
    data['Pred_down'].append(item[1][0][0])
    data['Pred_up'].append(item[1][0][1])
    data['Date'].append(item[2][0])

# Converted to Pandas data frame
df = pd.DataFrame(data)

In [96]:
def print_ext(updown, rate, data):
    UPtarget = 1 if updown == 'UP' else 0

    # Extract rows from the data frame that match the condition (Pred_zero or Pred_one exceeds the threshold)
    filtered_data = data[((data['Pred_down'] > rate) | (data['Pred_up'] > rate)) & (data['Label'] == UPtarget)]

    if UPtarget == 1:
      target = filtered_data['Pred_up']
    else:
      target = filtered_data['Pred_down']
    # Number of rows matching the condition that were correctly predicted
    correct_predictions = filtered_data[(target > rate)]

    counter = len(correct_predictions)

    total_counter = len(filtered_data)
    print(filtered_data)
    # Display the result
    if total_counter > 0:
        accuracy = counter / total_counter
        print('Accuracy: {:.4f}'.format(accuracy))
    else:
        print('Accuracy: N/A - division by zero')
    print('Number of correct answers within the predicted: {}'.format(counter))
    print('Number of {} exceeding the threshold {}: {}'.format(updown,rate, total_counter))
    print("--------------------------------------------------------------")



In [97]:
print_ext('UP', 0.5, df)
print_ext('DOWN', 0.5, df)
print_ext('UP', 0.7, df)
print_ext('DOWN', 0.7, df)
print_ext('UP', 0.8, df)
print_ext('DOWN', 0.8, df)
print_ext('UP', 0.9, df)
print_ext('DOWN', 0.9, df)

   Label  Pred_down   Pred_up        Date
2      1   0.000028  0.999972  2024-01-10
5      1   0.000050  0.999950  2024-01-15
6      1   0.074721  0.925278  2024-01-16
7      1   0.007780  0.992220  2024-01-17
Accuracy: 1.0000
Number of correct answers within the predicted: 4
Number of UP exceeding the threshold 0.5: 4
--------------------------------------------------------------
   Label  Pred_down   Pred_up        Date
0      0   0.924562  0.075438  2024-01-08
1      0   0.444574  0.555426  2024-01-09
3      0   0.747689  0.252311  2024-01-11
4      0   0.868400  0.131600  2024-01-12
8      0   0.959211  0.040789  2024-01-18
9      0   0.975627  0.024373  2024-01-19
Accuracy: 0.8333
Number of correct answers within the predicted: 5
Number of DOWN exceeding the threshold 0.5: 6
--------------------------------------------------------------
   Label  Pred_down   Pred_up        Date
2      1   0.000028  0.999972  2024-01-10
5      1   0.000050  0.999950  2024-01-15
6      1   0.074721 